### Jupyter Notebook for filtering Binary masks of coin for each image from the results of Step1-Running SAM2.1 on Faba bean images

#### Author: Harpreet Kaur Bargota



In [ ]:
import os
import cv2
import pandas as pd
from skimage import measure
from skimage.measure import regionprops_table

# ------------------ Paths ------------------
main_folder = '/output_SAM2.1'
output_mask_dir = '/pipeline/coin_masks'

# Create output directory if it does not exist
os.makedirs(output_mask_dir, exist_ok=True)

# List to store coin features
coin_total = []

# ------------------ Loop through folders ------------------
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)

    if not os.path.isdir(subfolder_path):
        continue

    metadata_file_path = os.path.join(subfolder_path, 'metadata.csv')

    if not os.path.exists(metadata_file_path):
        continue

    # Read metadata
    df_metadata = pd.read_csv(metadata_file_path)

    # Filter coin candidates
    df_metadata_coin = df_metadata[
        (df_metadata['bbox_x0'] >= 3000) &
        (df_metadata['area'] >= 200000)
    ]

    # Take the first coin mask only
    if df_metadata_coin.empty:
        continue

    mask_index = df_metadata_coin.index[0]
    mask_path = os.path.join(subfolder_path, f'{mask_index}.png')

    if not os.path.exists(mask_path):
        continue

    # ------------------ Load and process mask ------------------
    mask_coin = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Save final coin mask
    image_name = subfolder.replace('.JPG', '')
    output_mask_path = os.path.join(
        output_mask_dir,
        f'{image_name}_coin.png'
    )
    cv2.imwrite(output_mask_path, mask_coin)